In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
import time
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.support import expected_conditions as EC
import bs4
import re
from selenium.webdriver.common.keys import Keys

In [39]:
# Webdriver
def setup_driver():
    """"start the chrome driver and go to sahometraders.co.za/to-rent
    """
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.implicitly_wait(5)
    driver.get('https://www.sahometraders.co.za/to-rent')
    time.sleep(2)
    return driver
driver = setup_driver() 



<ipython-input-39-b25a0fc41d75>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [43]:
suburbs_url_dict = {
                    "Sea Point":"https://www.sahometraders.co.za/property-to-rent-in-sea-point-s11021",
                    "Gardens":"https://www.sahometraders.co.za/property-to-rent-in-gardens-s9145",
                    "Green Point": "https://www.sahometraders.co.za/property-to-rent-in-green-point-s11017"
                    }

In [40]:
suburbs= ("Sea Point",'Green Point',"Gardens")
# we already have the base urls so we move straight to making the soup 



In [44]:
base_url = suburbs_url_dict[suburbs[0]]
base_url



'https://www.sahometraders.co.za/property-to-rent-in-sea-point-s11021'

In [47]:
def make_soup(driver):
    
    soup = BeautifulSoup(driver.page_source,"html.parser")
    body_soup = soup.body
    return body_soup

In [73]:
body_soup = make_soup(driver)

In [62]:
def number_of_pages(body_soup:bs4.element.Tag) -> int:
    """"find the number of listed places in the search to figure out how many pages to crawl"""
    listingcount_options = body_soup.find("div",{'class':"sc_pageText"})
    
    
    split_counts = re.findall('\d+',listingcount_options.text)
    
    this_page = int(split_counts[1])
    total_listings = int(split_counts[-1])
    pages_to_search = -(-total_listings//this_page)
    return pages_to_search

In [63]:
page_count = number_of_pages(body_soup)
# add ?Page=2 to the end of the base url 


In [64]:
def make_subsequent_urls(page_count,base_url):
    urls_list = []
    for i in range(2,page_count+1):
        # i will be our page numbers to shoot into the url. 
        next_url = base_url + f"?Page={i}"
        urls_list.append(next_url) 
    return urls_list
urls_list = make_subsequent_urls(page_count=page_count,base_url=base_url)

In [65]:
urls_list # sweeeet 

['https://www.sahometraders.co.za/property-to-rent-in-sea-point-s11021?Page=2',
 'https://www.sahometraders.co.za/property-to-rent-in-sea-point-s11021?Page=3',
 'https://www.sahometraders.co.za/property-to-rent-in-sea-point-s11021?Page=4',
 'https://www.sahometraders.co.za/property-to-rent-in-sea-point-s11021?Page=5',
 'https://www.sahometraders.co.za/property-to-rent-in-sea-point-s11021?Page=6',
 'https://www.sahometraders.co.za/property-to-rent-in-sea-point-s11021?Page=7',
 'https://www.sahometraders.co.za/property-to-rent-in-sea-point-s11021?Page=8',
 'https://www.sahometraders.co.za/property-to-rent-in-sea-point-s11021?Page=9',
 'https://www.sahometraders.co.za/property-to-rent-in-sea-point-s11021?Page=10']

In [106]:
d = {'title':'','priceDescription':'','priceAdditionalDescriptor':'', 'suburb':'',
           'number_of_bedrooms':'','number_of_bathrooms':'','number_of_garages':'',"size":"",
           'available_from':'',"url":"",}
url_info =body_soup.find_all("div",{'class':"p24_regularTile"})
url_info[3].find('a')["href"]
"https://www.sahometraders.co.za" + url_info[3].find('a')["href"]

'https://www.sahometraders.co.za/commercial-property-to-rent-in-green-point-112231165'

In [107]:
import pandas as pd
def extract_listing_info(soup_body):
  """ Extract info from all listings on one page of property24.com/to-rent
      
  """
  data = []
    

  # FIND ALL listings 
  info_of_listings = soup_body.find_all(['div',"span","a"],{'class',"p24_content"}) # seems to be only span, but will just leave the divs and a there 
  image_info = soup_body.find_all(['span','div'],{"class":"js_listingTileImageHolder"}) # need to find a waty to get rid of the duplicate
  url_info =soup_body.find_all("div",{'class':"p24_regularTile"}) # get the href out of there
  # so far no promoted tiles, unlike P24
  print('len(info_of_listings)',len(info_of_listings),"len(image_info)",len(image_info),"len(url_info)",len(url_info))
  print('len(info_of_listings)',len(set(info_of_listings)),"len(image_info)",len(set(image_info)),"len(url_info)",len(set(url_info)))
  # so on page 2 of Greenpoint, suddenly there is one less in the info of listings. 
  # Which means it actually wasn't duplicated for once, but still it causes problems
  # we need to remove the duplicates earlier
  assert len(info_of_listings)== len(image_info)==len(url_info)
  print("BATCH SIZE:", len(info_of_listings))
  

  # Extracting to a dictionary
  counter = 0 
  for i,j,k in zip(info_of_listings,image_info,url_info): 
    # time will tell if the duplicate is giving us problems or if it's the same one every page then we can just delete it at the end 
      d = {'title':'','priceDescription':'','priceAdditionalDescriptor':'', 'suburb':'',
           'number_of_bedrooms':'','number_of_bathrooms':'','number_of_garages':'',"size":"",
           'available_from':'',"url":"",}  
           
      
      # we need an or for the available from or available now flags
      try:
        d['title']                      = i.find(['span'],{"class":'p24_propertyTitle'}).text 
        d['priceDescription']           =i.find('span',{"class":'p24_price'}).text.strip()
        if i.find('span',{"class":"left rentalTerm p24_rentalTerm"}): 
          d['priceAdditionalDescriptor']  = i.find(['div','span'],{"class":"left rentalTerm p24_rentalTerm"}).text
        else: 
          d['priceAdditionalDescriptor'] = None
          
        d['suburb']                     = i.find('span',{"class":"p24_location"}).text

        if i.find("span",{"class":"p24_featureDetails","title":"Bedrooms"}):
          d['number_of_bedrooms']         = i.find("span",{"class":"p24_featureDetails","title":"Bedrooms"}).text.strip()
        else: 
          d['number_of_bedrooms'] = None
        
        if i.find("span",{"class":"p24_featureDetails","title":"Bathrooms"}):
          d['number_of_bathrooms']        = i.find("span",{"class":"p24_featureDetails","title":"Bathrooms"}).text.strip()
        else: d['number_of_bathrooms'] = None
        
        if i.find("span",{"class":"p24_featureDetails","title":"Parking Spaces"}):
          d['number_of_garages']          = i.find("span",{"class":"p24_featureDetails","title":"Parking Spaces"}).text.strip()
        else: d['number_of_garages'] = None
        
        if i.find("span",{"class":"p24_size"}):
          d['size'] = i.find("span",{"class":"p24_size"}).text.strip() # floor size for apartments and houses, commerical property sometimes has
                                                                      #floor size and sometimes has gross-lettable-area. could split in future if needed
        else:
          d["size"] = None 

        if j.find('li',{"class":"p24_availableBadge"}):
          d['available_from']   = j.find('li',{"class":"p24_availableBadge"}).text
        else:
          d['available_from'] = None
        
        

        
        
        d['url']              = "https://www.sahometraders.co.za" + url_info[3].find('a')["href"]
         
        
      except Exception as e: 
        print(e)
      data.append(d) 
      counter +=1  
  return data 

In [113]:
def main():
    #steps
    all_data = []
    # go to url and get throught the cookie
    driver  = setup_driver()
    # choose the suburb from a list 
    suburbs= ("Sea Point",'Green Point',"Gardens")
    suburbs_url_dict = {
                    "Sea Point":"https://www.sahometraders.co.za/property-to-rent-in-sea-point-s11021",
                    "Gardens":"https://www.sahometraders.co.za/property-to-rent-in-gardens-s9145",
                    "Green Point": "https://www.sahometraders.co.za/property-to-rent-in-green-point-s11017"
                    }
    for suburb in suburbs:
        base_url = suburbs_url_dict[suburb]
        time.sleep(4)
        driver.get(base_url)
        soup_body = make_soup(driver) 
        number_of_pages_to_search = number_of_pages(soup_body)
        other_urls_list = make_subsequent_urls(number_of_pages_to_search,base_url)

        #get info from one page 
        data = extract_listing_info(soup_body)
        all_data.extend(data)
        
        for url in other_urls_list:
            driver.get(url)
            next_soup_body  = make_soup(driver)
            data_next = extract_listing_info(next_soup_body)
            all_data.extend(data_next)
        
        time.sleep(5)
        
        
    driver.close()
    df = pd.DataFrame(all_data,index = range(len(all_data)))
    
    return df

In [114]:
df = main()

<ipython-input-39-b25a0fc41d75>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


len(info_of_listings) 21 len(image_info) 21 len(url_info) 21
len(info_of_listings) 21 len(image_info) 20 len(url_info) 21
BATCH SIZE: 21
len(info_of_listings) 21 len(image_info) 21 len(url_info) 21
len(info_of_listings) 21 len(image_info) 20 len(url_info) 21
BATCH SIZE: 21
len(info_of_listings) 21 len(image_info) 21 len(url_info) 21
len(info_of_listings) 21 len(image_info) 16 len(url_info) 21
BATCH SIZE: 21
len(info_of_listings) 21 len(image_info) 21 len(url_info) 21
len(info_of_listings) 21 len(image_info) 20 len(url_info) 21
BATCH SIZE: 21
len(info_of_listings) 21 len(image_info) 21 len(url_info) 21
len(info_of_listings) 21 len(image_info) 20 len(url_info) 21
BATCH SIZE: 21
len(info_of_listings) 21 len(image_info) 21 len(url_info) 21
len(info_of_listings) 21 len(image_info) 19 len(url_info) 21
BATCH SIZE: 21
len(info_of_listings) 21 len(image_info) 21 len(url_info) 21
len(info_of_listings) 21 len(image_info) 20 len(url_info) 21
BATCH SIZE: 21
len(info_of_listings) 21 len(image_info) 

In [116]:
df.tail(5)
# sweet

,title,priceDescription,priceAdditionalDescriptor,suburb,number_of_bedrooms,number_of_bathrooms,number_of_garages,size,available_from,url
673,Commercial Property,R 11 500,None,Gardens,None,None,None,89 m²,AVAILABLE NOW,https://www.sahometraders.co.za/commercial-pro...
674,Commercial Property,R 51 216,None,Gardens,None,None,None,338 m²,AVAILABLE NOW,https://www.sahometraders.co.za/commercial-pro...
675,Commercial Property,R 7 140,None,Gardens,None,None,None,102 m²,AVAILABLE NOW,https://www.sahometraders.co.za/commercial-pro...
676,Commercial Property,R 7 326,None,Gardens,None,None,None,37 m²,AVAILABLE NOW,https://www.sahometraders.co.za/commercial-pro...
677,Commercial Property,R 9 240,None,Gardens,None,None,None,132 m²,AVAILABLE NOW,https://www.sahometraders.co.za/commercial-pro...
